# Escala de grises

In [ ]:
import os
from PIL import Image

In [ ]:
# Definir las rutas de la carpeta origen y destino
src_folder = r"C:\Users\juanj\Desktop\DATA FINAL\Final"
dst_folder = r"C:\Users\juanj\Desktop\DATA FINAL\Grises"

In [ ]:
# Crear la carpeta destino si no existe
if not os.path.exists(dst_folder):
    os.makedirs(dst_folder)

# Recorrer la carpeta origen de forma recursiva
for root, dirs, files in os.walk(src_folder):
    # Obtener la ruta relativa respecto a la carpeta origen para mantener la misma estructura
    rel_path = os.path.relpath(root, src_folder)
    # La subcarpeta (label) será el nombre de la carpeta actual (o su subestructura)
    dst_subfolder = os.path.join(dst_folder, rel_path)
    
    # Crear la subcarpeta en el destino si no existe
    if not os.path.exists(dst_subfolder):
        os.makedirs(dst_subfolder)
    print(f"Creando carpeta: {dst_subfolder}")
    # Procesar cada archivo en la carpeta actual
    for file in files:
        # Opcional: comprobar que el archivo tenga extensión de imagen
        if file.lower().endswith(('.jpg','.jpeg','.png','.bmp')):
            src_file_path = os.path.join(root, file)
            dst_file_path = os.path.join(dst_subfolder, file)
            try:
                # Abrir la imagen, convertirla a escala de grises y guardarla
                img = Image.open(src_file_path)
                img_gray = img.convert('L')  # 'L' indica modo de escala de grises
                img_gray.save(dst_file_path)
            except Exception as e:
                print(f"Error procesando {src_file_path}: {e}")


# Ruido

In [ ]:
import os
import random
from PIL import Image, ImageFilter

In [ ]:
# Rutas de origen y destino
src_folder = r"C:\Users\juanj\Desktop\DATA FINAL\Grises"
dst_folder = r"C:\Users\juanj\Desktop\DATA FINAL\Ruido"

In [ ]:
# Crear carpeta destino si no existe
if not os.path.exists(dst_folder):
    os.makedirs(dst_folder)

# Función para crop central
def central_crop(img, scale=0.5):
    width, height = img.size
    new_width = int(width * scale)
    new_height = int(height * scale)
    left = (width - new_width) // 2
    top = (height - new_height) // 2
    return img.crop((left, top, left + new_width, top + new_height))

# Función para crop aleatorio
def random_crop(img, scale):
    width, height = img.size
    new_width = int(width * scale)
    new_height = int(height * scale)
    max_left = width - new_width
    max_top = height - new_height
    left = random.randint(0, max_left) if max_left > 0 else 0
    top = random.randint(0, max_top) if max_top > 0 else 0
    return img.crop((left, top, left + new_width, top + new_height))

In [ ]:
# Ángulos de rotación a aplicar
rotation_angles = [0, 90, 180, 270]

# Niveles de blur (porcentajes) y sufijos para el nombre
blur_levels = [(0.001, "blur1"), (0.002, "blur2")]

# Recorrer la carpeta origen manteniendo la estructura
for root, dirs, files in os.walk(src_folder):
    # Ruta relativa para preservar la estructura de subcarpetas
    rel_path = os.path.relpath(root, src_folder)
    dst_subfolder = os.path.join(dst_folder, rel_path)
    if not os.path.exists(dst_subfolder):
        os.makedirs(dst_subfolder)
    print(rel_path)
    for file in files:
        if file.lower().endswith(('.jpg','.jpeg','.png','.bmp')):
            src_path = os.path.join(root, file)
            try:
                # Abrir imagen
                img = Image.open(src_path)
                
                # Guardar la imagen original en destino
                dst_original = os.path.join(dst_subfolder, file)
                img.save(dst_original)
                
                # Iterar por cada ángulo de rotación
                for angle in rotation_angles:
                    # Para ángulo 0 usamos la imagen original
                    if angle == 0:
                        rotated_img = img
                    else:
                        rotated_img = img.rotate(angle, expand=True)
                        # Guardar la imagen rotada (excepto 0 para evitar duplicados de la original)
                        base_name, ext = os.path.splitext(file)
                        rot_filename = f"{base_name}_rot{angle}{ext}"
                        dst_rot_path = os.path.join(dst_subfolder, rot_filename)
                        rotated_img.save(dst_rot_path)
                    
                    # --- Generar variantes de recorte ---
                    # Crop central (factor 0.5)
                    crop_central = central_crop(rotated_img, scale=0.5)
                    central_name = f"{os.path.splitext(file)[0]}_rot{angle}_crop_central{os.path.splitext(file)[1]}"
                    central_path = os.path.join(dst_subfolder, central_name)
                    crop_central.save(central_path)
                    
                    # 3 crops aleatorios
                    for i in range(1, 4):
                        random_scale = random.uniform(0.4, 0.6)
                        crop_rand = random_crop(rotated_img, scale=random_scale)
                        rand_name = f"{os.path.splitext(file)[0]}_rot{angle}_crop_rand{i}{os.path.splitext(file)[1]}"
                        rand_path = os.path.join(dst_subfolder, rand_name)
                        crop_rand.save(rand_path)
                    
                    # --- Generar variantes con blur ---
                    # Se calcula el radio de blur basado en el lado menor de la imagen
                    width, height = rotated_img.size
                    for perc, suffix in blur_levels:
                        radius = max(int(min(width, height) * perc), 1)
                        blurred_img = rotated_img.filter(ImageFilter.GaussianBlur(radius))
                        blur_name = f"{os.path.splitext(file)[0]}_rot{angle}_{suffix}{os.path.splitext(file)[1]}"
                        blur_path = os.path.join(dst_subfolder, blur_name)
                        blurred_img.save(blur_path)
                
                #print(f"Procesada: {src_path}")
            except Exception as e:
                print(f"Error procesando {src_path}: {e}")


# Fast Fourier Transform r=30

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Ruta de la carpeta raíz que contiene las subcarpetas con imágenes
root_dir = r"G:\Ruido"  # Cambia esta ruta según corresponda

# Valor del radio para el filtro pasa alto (ajusta según necesites)
radius = 30

def process_image(image_path, radius):
    # Abrir la imagen y convertirla a escala de grises
    img = Image.open(image_path).convert("L")
    img_array = np.array(img)
    
    # --- 1) Calcular FFT y obtener la magnitud ---
    fft = np.fft.fft2(img_array)
    fft_shift = np.fft.fftshift(fft)
    magnitude_spectrum = 20 * np.log(np.abs(fft_shift) + 1)
    
    # Normalizar la magnitud a 0-255 para guardarla correctamente
    mag_norm = (magnitude_spectrum - magnitude_spectrum.min()) / (magnitude_spectrum.max() - magnitude_spectrum.min()) * 255
    fft_image = Image.fromarray(np.uint8(mag_norm))
    
    # Guardar imagen FFT en la misma subcarpeta con sufijo _fft
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    dirpath = os.path.dirname(image_path)
    fft_image_path = os.path.join(dirpath, f"{base_name}_fft.png")
    fft_image.save(fft_image_path)
    
    # --- 2) Aplicar filtro pasa alto y calcular la inversa ---
    rows, cols = img_array.shape
    crow, ccol = rows // 2, cols // 2
    y, x = np.ogrid[:rows, :cols]
    # Máscara circular invertida para el filtro pasa alto
    mask = (x - ccol)**2 + (y - crow)**2 >= radius**2
    
    # Aplicar la máscara al espectro
    fft_shift_hp = fft_shift * mask
    ifft_shift = np.fft.ifftshift(fft_shift_hp)
    img_filtered = np.fft.ifft2(ifft_shift)
    img_filtered = np.abs(img_filtered)
    
    # Normalizar la imagen filtrada a 0-255
    img_filtered_norm = (img_filtered - img_filtered.min()) / (img_filtered.max() - img_filtered.min()) * 255
    filtered_image = Image.fromarray(np.uint8(img_filtered_norm))
    
    # Guardar imagen filtrada en la misma subcarpeta con sufijo _highpass
    filtered_image_path = os.path.join(dirpath, f"{base_name}_highpass.png")
    filtered_image.save(filtered_image_path)

# Recorrer la carpeta raíz y sus subcarpetas
for dirpath, dirnames, filenames in os.walk(root_dir):
    # Filtrar archivos que sean imágenes (ajusta las extensiones si es necesario)
    image_files = [f for f in filenames if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    print(f"Procesando carpeta: {dirpath}")
    for image_file in image_files:
        image_path = os.path.join(dirpath, image_file)
        process_image(image_path, radius)


# Transformada y LBP

In [ ]:
import os
import json
import numpy as np
import cv2
from skimage.feature import local_binary_pattern

In [ ]:
# Carpeta raíz
root_folder = r"C:\Users\juanj\Desktop\DATA FINAL\Ruido"

In [ ]:
# Parámetros para LBP
radius = 1
n_points = 8 * radius  # Ejemplo: 8 vecinos
METHOD = 'uniform'

# Recorre cada subcarpeta dentro de la carpeta raíz
for subdir, dirs, files in os.walk(root_folder):
    # Filtrar archivos de imagen
    image_files = [f for f in files if f.lower().endswith(('.jpg'))]
    if not image_files:
        continue  # Si no hay imágenes en esta carpeta, se salta

    # Diccionario para guardar los resultados
    results = {}
    print(f"Procesando carpeta: {subdir}")
    
    # Procesar cada imagen en la subcarpeta
    for image_file in image_files:
        image_path = os.path.join(subdir, image_file)
        # Leer la imagen en escala de grises
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"No se pudo leer la imagen {image_path}")
            continue
        
        # --- Transformada de Fourier ---
        # Se calcula la FFT, se centra y se obtiene el espectro de magnitud
        f = np.fft.fft2(img)
        fshift = np.fft.fftshift(f)
        magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1)  
        # Guardar el array de magnitud de Fourier sin normalización ni conversión
        fourier_filename = os.path.splitext(image_file)[0] + "_fourier.npy"
        fourier_path = os.path.join(subdir, fourier_filename)
        np.save(fourier_path, magnitude_spectrum)
        
        # --- Local Binary Pattern (LBP) ---
        lbp = local_binary_pattern(img, n_points, radius, METHOD)
        # Guardar el array de LBP sin normalización ni conversión
        lbp_filename = os.path.splitext(image_file)[0] + "_lbp.npy"
        lbp_path = os.path.join(subdir, lbp_filename)
        np.save(lbp_path, lbp)
        
        # Guardar la relación en el diccionario
        results[image_file] = {
            "fourier": fourier_filename,
            "lbp": lbp_filename
        }
    
    # Guardar el diccionario en un archivo JSON dentro de la misma subcarpeta
    json_path = os.path.join(subdir, "resultados.json")
    with open(json_path, "w") as json_file:
        json.dump(results, json_file, indent=4)
